## Initializing


In [1]:
from pyspark import SparkConf, SparkContext

spark_conf = SparkConf()\
        .setAppName("Assignment 1")
sc=SparkContext.getOrCreate(spark_conf) 

input_file='AllVideos.csv'

videoData = sc.textFile('AllVideos.csv')


## 3.1 Controversial Trending Videos Identification

In [21]:
def extractData(record):
    try:
        videoId, date, category, views, likes, dislikes, country = record.split(",")
        likes = int(likes)
        dislikes= int (dislikes)
        dateList = date.split(".")
        dateList[1], dateList[2] = dateList[2],dateList[1]

        dateString= ".".join(dateList)

        return((videoId, country), (dateString, likes, dislikes))
    except:
        return ()

        

def dislikeRate(value):
    vlist = list(value)
    listLength = len(vlist)
    if(listLength<2):
        return ""
    else:  
        return (vlist[listLength-1][2] - vlist[0][2]) - (vlist[listLength-1][1] - vlist[0][1])
        


videoData.map(extractData).sortBy(lambda row: row[1][0], ascending=True).groupByKey().mapValues(dislikeRate) \
.filter(lambda row :isinstance(row[1], int)) \
.sortBy(lambda row: row[1], ascending=False) \
.map(lambda row: (row[0][0], row[1], row[0][1])).take(10)





[('QwZT7T-TXT0', 579119, 'GB'),
 ('QwZT7T-TXT0', 478100, 'US'),
 ('BEePFpC9qG8', 365862, 'DE'),
 ('RmZ3DPJQo2k', 334390, 'KR'),
 ('q8v9MvManKE', 299044, 'IN'),
 ('pOHQdIDds6s', 160365, 'CA'),
 ('ZGEoqPpJQLE', 151913, 'RU'),
 ('84LBjXaeKk4', 134836, 'FR'),
 ('84LBjXaeKk4', 134834, 'DE'),
 ('84LBjXaeKk4', 121240, 'RU')]

## 3.2 Category and Trending Correlation

In [23]:
def extractCountry(record):
    try:
        videoId, date, category, views, likes, dislikes, country = record.split(",")
        return(videoId,country )
    except:
        return ()
    
def extractCategory(record):
    try:
        videoId, date, category, views, likes, dislikes, country = record.split(",")
        return(videoId, category)
    except:
        return ()
    
def avg(values):
    vlist = list(values)
    return sum(vlist)/len(vlist)

videoCatogory= videoData.map(extractCategory).distinct()

videoCountry = videoData.map(extractCountry).distinct().groupByKey().mapValues(len)

videoCatogory.join(videoCountry).values().groupByKey() \
.mapValues(avg).sortBy(lambda row:row[1],ascending=True).collect()

[('Trailers', 1.0),
 ('Autos & Vehicles', 1.0190448285965426),
 ('News & Politics', 1.052844979051223),
 ('Nonprofits & Activism', 1.057344064386318),
 ('Education', 1.0628976994615762),
 ('People & Blogs', 1.0640343760329336),
 ('Pets & Animals', 1.0707850707850708),
 ('Howto & Style', 1.0876256925918326),
 ('Travel & Events', 1.0929411764705883),
 ('Gaming', 1.0946163477016635),
 ('Sports', 1.1422245184146431),
 ('Entertainment', 1.1447534885477444),
 ('Science & Technology', 1.1626835588828102),
 ('Film & Animation', 1.1677314564158094),
 ('Comedy', 1.2144120659156503),
 ('Movies', 1.25),
 ('Music', 1.310898044427568),
 ('Shows', 1.614678899082569)]